Primeiro, mudo o working directory para um nível anterior para poder utilizar o módulo dashboard

In [1]:
%cd '..'

/git/hydra


/home/vagrant/python_envs/jupyter/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import geopandas as gpd
from dotenv import load_dotenv
import duckdb
import os
from pyarrow.fs import S3FileSystem
import folium

from dashboard.dao import DuckDBDAO

In [3]:
load_dotenv()

True

## Setup de variáveis iniciais e conexão do duckdb

In [4]:
AWS_S3_BUCKET = os.getenv("MINIO_SILVER_BUCKET_NAME")
AWS_ACCESS_KEY_ID = os.getenv("MINIO_ROOT_USER")
AWS_SECRET_ACCESS_KEY = os.getenv("MINIO_ROOT_PASSWORD")
ENDPOINT_OVERRIDE = os.getenv("MINIO_ENDPOINT_URL")

In [5]:
duckdb_dao = DuckDBDAO(
    bucket_name=AWS_S3_BUCKET,
    access_key=AWS_ACCESS_KEY_ID,
    secret_key=AWS_SECRET_ACCESS_KEY,
    endpoint=ENDPOINT_OVERRIDE
)

load_parquet = duckdb_dao.load_parquet

## Carregar dados do asset `setor_censitario_enriched`

In [6]:
gdf_setor = load_parquet('setor_censitario_enriched')

Gerando query para o arquivo s3://silver-db/dagster/setor_censitario_enriched.parquet...
Query gerada:
SELECT * EXCLUDE (geometry), ST_AsText(ST_GeomFromWKB(geometry)) AS geometry FROM read_parquet("s3://silver-db/dagster/setor_censitario_enriched.parquet")


In [7]:
gdf_setor

,cd_original_setor_censitario,Cod_setor,Domicilio01_SP1_V001,Domicilio01_SP1_V002,Domicilio01_SP1_V012,Domicilio01_SP1_V013,Domicilio01_SP1_V014,Domicilio01_SP1_V017,Domicilio01_SP1_V018,Domicilio01_SP1_V019,Domicilio01_SP1_V020,Domicilio01_SP1_V021,Domicilio01_SP1_V035,Domicilio01_SP1_V039,Domicilio01_SP1_V040,Domicilio01_SP1_V041,missing,supressed,geometry
0,355030801000001,355030801000001,268.0,266.0,266.0,0.0,0.0,266.0,0.0,0.0,0.0,0.0,265.0,0.0,1.0,0.0,False,False,"POLYGON ((-46.56957 -23.56917, -46.57010 -23.5..."
1,355030801000002,355030801000002,306.0,306.0,306.0,0.0,0.0,306.0,0.0,0.0,0.0,0.0,306.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.56876 -23.56854, -46.56883 -23.5..."
2,355030801000003,355030801000003,189.0,189.0,189.0,0.0,0.0,187.0,1.0,1.0,0.0,0.0,189.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.56769 -23.57054, -46.56780 -23.5..."
3,355030801000004,355030801000004,181.0,181.0,181.0,0.0,0.0,181.0,0.0,0.0,0.0,0.0,181.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.56747 -23.57330, -46.56821 -23.5..."
4,355030801000005,355030801000005,240.0,240.0,240.0,0.0,0.0,240.0,0.0,0.0,0.0,0.0,240.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.57120 -23.57313, -46.57127 -23.5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18948,355030896000243,355030896000243,96.0,96.0,96.0,0.0,0.0,96.0,0.0,0.0,0.0,0.0,96.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.38858 -23.54386, -46.38858 -23.5..."
18949,355030896000244,355030896000244,54.0,54.0,54.0,0.0,0.0,52.0,0.0,0.0,0.0,2.0,54.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.38815 -23.54508, -46.38816 -23.5..."
18950,355030896000245,355030896000245,87.0,87.0,87.0,0.0,0.0,15.0,0.0,0.0,0.0,72.0,87.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.42368 -23.52080, -46.42347 -23.5..."
18951,355030896000246,355030896000246,38.0,38.0,38.0,0.0,0.0,18.0,0.0,0.0,0.0,20.0,38.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.42247 -23.52123, -46.42214 -23.5..."


## Classificar aleatoriamente os setores em 5 categorias, nas seguintes proporções

1. 10% dos setores como high-high;

2. 
10% dos setores como low-low

3. 
5% dos setores como high-lo

4. ;
5% dos setores como low-hi

5. h;
70% dos setores como non-significant.

In [8]:
gdf_setor_lisa = gdf_setor.copy()
gdf_setor_lisa['lisa_class'] = 'non-significant'
gdf_setor_lisa

,cd_original_setor_censitario,Cod_setor,Domicilio01_SP1_V001,Domicilio01_SP1_V002,Domicilio01_SP1_V012,Domicilio01_SP1_V013,Domicilio01_SP1_V014,Domicilio01_SP1_V017,Domicilio01_SP1_V018,Domicilio01_SP1_V019,Domicilio01_SP1_V020,Domicilio01_SP1_V021,Domicilio01_SP1_V035,Domicilio01_SP1_V039,Domicilio01_SP1_V040,Domicilio01_SP1_V041,missing,supressed,geometry,lisa_class
0,355030801000001,355030801000001,268.0,266.0,266.0,0.0,0.0,266.0,0.0,0.0,0.0,0.0,265.0,0.0,1.0,0.0,False,False,"POLYGON ((-46.56957 -23.56917, -46.57010 -23.5...",non-significant
1,355030801000002,355030801000002,306.0,306.0,306.0,0.0,0.0,306.0,0.0,0.0,0.0,0.0,306.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.56876 -23.56854, -46.56883 -23.5...",non-significant
2,355030801000003,355030801000003,189.0,189.0,189.0,0.0,0.0,187.0,1.0,1.0,0.0,0.0,189.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.56769 -23.57054, -46.56780 -23.5...",non-significant
3,355030801000004,355030801000004,181.0,181.0,181.0,0.0,0.0,181.0,0.0,0.0,0.0,0.0,181.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.56747 -23.57330, -46.56821 -23.5...",non-significant
4,355030801000005,355030801000005,240.0,240.0,240.0,0.0,0.0,240.0,0.0,0.0,0.0,0.0,240.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.57120 -23.57313, -46.57127 -23.5...",non-significant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18948,355030896000243,355030896000243,96.0,96.0,96.0,0.0,0.0,96.0,0.0,0.0,0.0,0.0,96.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.38858 -23.54386, -46.38858 -23.5...",non-significant
18949,355030896000244,355030896000244,54.0,54.0,54.0,0.0,0.0,52.0,0.0,0.0,0.0,2.0,54.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.38815 -23.54508, -46.38816 -23.5...",non-significant
18950,355030896000245,355030896000245,87.0,87.0,87.0,0.0,0.0,15.0,0.0,0.0,0.0,72.0,87.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.42368 -23.52080, -46.42347 -23.5...",non-significant
18951,355030896000246,355030896000246,38.0,38.0,38.0,0.0,0.0,18.0,0.0,0.0,0.0,20.0,38.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.42247 -23.52123, -46.42214 -23.5...",non-significant


In [9]:
sample_pct = 0.1
sample = gdf_setor_lisa.sample(frac=sample_pct)
sample

,cd_original_setor_censitario,Cod_setor,Domicilio01_SP1_V001,Domicilio01_SP1_V002,Domicilio01_SP1_V012,Domicilio01_SP1_V013,Domicilio01_SP1_V014,Domicilio01_SP1_V017,Domicilio01_SP1_V018,Domicilio01_SP1_V019,Domicilio01_SP1_V020,Domicilio01_SP1_V021,Domicilio01_SP1_V035,Domicilio01_SP1_V039,Domicilio01_SP1_V040,Domicilio01_SP1_V041,missing,supressed,geometry,lisa_class
8513,355030843000043,355030843000043,138.0,138.0,138.0,0.0,0.0,137.0,1.0,0.0,0.0,0.0,138.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.78444 -23.69682, -46.78444 -23.6...",non-significant
802,355030808000017,355030808000017,353.0,294.0,294.0,0.0,0.0,294.0,0.0,0.0,0.0,0.0,294.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.59896 -23.53810, -46.59890 -23.5...",non-significant
14843,355030875000032,355030875000032,244.0,244.0,244.0,0.0,0.0,236.0,5.0,3.0,0.0,0.0,244.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.46211 -23.63213, -46.46214 -23.6...",non-significant
12885,355030865000155,355030865000155,86.0,86.0,86.0,0.0,0.0,86.0,0.0,0.0,0.0,0.0,86.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.78806 -23.59840, -46.78822 -23.5...",non-significant
14908,355030875000097,355030875000097,221.0,221.0,221.0,0.0,0.0,219.0,2.0,0.0,0.0,0.0,221.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.44795 -23.62456, -46.44793 -23.6...",non-significant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3751,355030823000048,355030823000048,399.0,399.0,399.0,0.0,0.0,394.0,4.0,0.0,0.0,0.0,397.0,0.0,2.0,0.0,False,False,"POLYGON ((-46.70427 -23.72926, -46.70416 -23.7...",non-significant
14104,355030870000221,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,"POLYGON ((-46.61186 -23.49052, -46.61167 -23.4...",non-significant
918,355030810000004,355030810000004,583.0,583.0,583.0,0.0,0.0,583.0,0.0,0.0,0.0,0.0,583.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.61786 -23.54372, -46.61781 -23.5...",non-significant
4796,355030827000184,355030827000184,85.0,85.0,85.0,0.0,0.0,85.0,0.0,0.0,0.0,0.0,85.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.62301 -23.62928, -46.62308 -23.6...",non-significant


In [10]:
sample_filter = gdf_setor_lisa['cd_original_setor_censitario'].isin(sample['cd_original_setor_censitario'])
gdf_setor_lisa.loc[sample_filter, 'lisa_class'] = 'high-high'
gdf_setor_lisa

,cd_original_setor_censitario,Cod_setor,Domicilio01_SP1_V001,Domicilio01_SP1_V002,Domicilio01_SP1_V012,Domicilio01_SP1_V013,Domicilio01_SP1_V014,Domicilio01_SP1_V017,Domicilio01_SP1_V018,Domicilio01_SP1_V019,Domicilio01_SP1_V020,Domicilio01_SP1_V021,Domicilio01_SP1_V035,Domicilio01_SP1_V039,Domicilio01_SP1_V040,Domicilio01_SP1_V041,missing,supressed,geometry,lisa_class
0,355030801000001,355030801000001,268.0,266.0,266.0,0.0,0.0,266.0,0.0,0.0,0.0,0.0,265.0,0.0,1.0,0.0,False,False,"POLYGON ((-46.56957 -23.56917, -46.57010 -23.5...",non-significant
1,355030801000002,355030801000002,306.0,306.0,306.0,0.0,0.0,306.0,0.0,0.0,0.0,0.0,306.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.56876 -23.56854, -46.56883 -23.5...",non-significant
2,355030801000003,355030801000003,189.0,189.0,189.0,0.0,0.0,187.0,1.0,1.0,0.0,0.0,189.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.56769 -23.57054, -46.56780 -23.5...",non-significant
3,355030801000004,355030801000004,181.0,181.0,181.0,0.0,0.0,181.0,0.0,0.0,0.0,0.0,181.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.56747 -23.57330, -46.56821 -23.5...",non-significant
4,355030801000005,355030801000005,240.0,240.0,240.0,0.0,0.0,240.0,0.0,0.0,0.0,0.0,240.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.57120 -23.57313, -46.57127 -23.5...",non-significant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18948,355030896000243,355030896000243,96.0,96.0,96.0,0.0,0.0,96.0,0.0,0.0,0.0,0.0,96.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.38858 -23.54386, -46.38858 -23.5...",non-significant
18949,355030896000244,355030896000244,54.0,54.0,54.0,0.0,0.0,52.0,0.0,0.0,0.0,2.0,54.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.38815 -23.54508, -46.38816 -23.5...",non-significant
18950,355030896000245,355030896000245,87.0,87.0,87.0,0.0,0.0,15.0,0.0,0.0,0.0,72.0,87.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.42368 -23.52080, -46.42347 -23.5...",non-significant
18951,355030896000246,355030896000246,38.0,38.0,38.0,0.0,0.0,18.0,0.0,0.0,0.0,20.0,38.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.42247 -23.52123, -46.42214 -23.5...",non-significant


In [11]:
gdf_setor_lisa[sample_filter]

,cd_original_setor_censitario,Cod_setor,Domicilio01_SP1_V001,Domicilio01_SP1_V002,Domicilio01_SP1_V012,Domicilio01_SP1_V013,Domicilio01_SP1_V014,Domicilio01_SP1_V017,Domicilio01_SP1_V018,Domicilio01_SP1_V019,Domicilio01_SP1_V020,Domicilio01_SP1_V021,Domicilio01_SP1_V035,Domicilio01_SP1_V039,Domicilio01_SP1_V040,Domicilio01_SP1_V041,missing,supressed,geometry,lisa_class
9,355030801000010,355030801000010,218.0,218.0,218.0,0.0,0.0,218.0,0.0,0.0,0.0,0.0,218.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.57314 -23.56783, -46.57353 -23.5...",high-high
13,355030801000014,355030801000014,311.0,311.0,311.0,0.0,0.0,311.0,0.0,0.0,0.0,0.0,311.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.56192 -23.56687, -46.56115 -23.5...",high-high
17,355030801000018,355030801000018,211.0,211.0,211.0,0.0,0.0,211.0,0.0,0.0,0.0,0.0,211.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.56061 -23.57302, -46.56084 -23.5...",high-high
19,355030801000020,355030801000020,249.0,249.0,249.0,0.0,0.0,245.0,0.0,1.0,0.0,2.0,249.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.56699 -23.57605, -46.56703 -23.5...",high-high
61,355030801000062,355030801000062,445.0,445.0,445.0,0.0,0.0,445.0,0.0,0.0,0.0,0.0,445.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.56993 -23.55762, -46.56871 -23.5...",high-high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18920,355030896000215,355030896000215,114.0,113.0,113.0,0.0,0.0,113.0,0.0,0.0,0.0,0.0,113.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.39836 -23.53641, -46.39772 -23.5...",high-high
18933,355030896000228,355030896000228,153.0,153.0,153.0,0.0,0.0,105.0,1.0,0.0,0.0,47.0,153.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.39082 -23.52447, -46.39091 -23.5...",high-high
18934,355030896000229,355030896000229,144.0,144.0,142.0,2.0,0.0,144.0,0.0,0.0,0.0,0.0,144.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.42762 -23.52655, -46.42738 -23.5...",high-high
18937,355030896000232,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,"POLYGON ((-46.39612 -23.52478, -46.39616 -23.5...",high-high


In [12]:
# TODO: repetir a mesma estrutura de maneira iterativa para todas as classificações

## Selecionar aleatoriamente um distrito para a exibição (alterando a caixa de seleção)

In [13]:
gdf_setor_distrito = load_parquet('intersection_setor_distrito_municipal', geometry_columns=['geometry', 'right_geometry'])

Gerando query para o arquivo s3://silver-db/dagster/intersection_setor_distrito_municipal.parquet...
Query gerada:
SELECT * EXCLUDE (geometry, right_geometry), ST_AsText(ST_GeomFromWKB(geometry)) AS geometry, ST_AsText(ST_GeomFromWKB(right_geometry)) AS right_geometry FROM read_parquet("s3://silver-db/dagster/intersection_setor_distrito_municipal.parquet")


In [14]:
gdf_setor_distrito

,cd_original_setor_censitario,negative_buffer,cd_identificador_distrito,cd_distrito_municipal,cd_identificador_subprefeitura,nm_distrito_municipal,sg_distrito_municipal,intersection,intersection_pct,__index_level_0__,geometry,right_geometry
0,355030801000001,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 59, 0, 0, 0, 126, ...",8583441.0,1,25.0,AGUA RASA,ARA,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 22, 0, 0, 0, 117, ...",1.000000,0,"POLYGON ((-46.56957 -23.56917, -46.57010 -23.5...","POLYGON ((341093.38042237 7391912.49139969, 34..."
1,355030801000002,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 58, 0, 0, 0, 65, 1...",8583441.0,1,25.0,AGUA RASA,ARA,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 33, 0, 0, 0, 66, 1...",1.000000,1,"POLYGON ((-46.56876 -23.56854, -46.56883 -23.5...","POLYGON ((341093.38042237 7391912.49139969, 34..."
2,355030801000003,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 96, 0, 0, 0, 136, ...",8583441.0,1,25.0,AGUA RASA,ARA,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 55, 0, 0, 0, 61, 1...",1.000000,2,"POLYGON ((-46.56769 -23.57054, -46.56780 -23.5...","POLYGON ((341093.38042237 7391912.49139969, 34..."
3,355030801000004,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 102, 0, 0, 0, 39, ...",8583441.0,1,25.0,AGUA RASA,ARA,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 64, 0, 0, 0, 223, ...",1.000000,3,"POLYGON ((-46.56747 -23.57330, -46.56821 -23.5...","POLYGON ((341093.38042237 7391912.49139969, 34..."
4,355030801000005,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 108, 0, 0, 0, 236,...",8583441.0,1,25.0,AGUA RASA,ARA,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 50, 0, 0, 0, 217, ...",1.000000,4,"POLYGON ((-46.57120 -23.57313, -46.57127 -23.5...","POLYGON ((341093.38042237 7391912.49139969, 34..."
...,...,...,...,...,...,...,...,...,...,...,...,...
18358,355030896000243,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 46, 0, 0, 0, 125, ...",8583486.0,96,28.0,LAJEADO,LAJ,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 78, 0, 0, 0, 207, ...",0.927643,18948,"POLYGON ((-46.38858 -23.54386, -46.38858 -23.5...","POLYGON ((357579.81520484 7394795.98601404, 35..."
18359,355030896000244,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 52, 0, 0, 0, 86, 1...",8583486.0,96,28.0,LAJEADO,LAJ,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 77, 0, 0, 0, 108, ...",0.961410,18949,"POLYGON ((-46.38815 -23.54508, -46.38816 -23.5...","POLYGON ((357579.81520484 7394795.98601404, 35..."
18360,355030896000245,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 18, 0, 0, 0, 106, ...",8583486.0,96,28.0,LAJEADO,LAJ,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 15, 0, 0, 0, 70, 6...",1.000000,18950,"POLYGON ((-46.42368 -23.52080, -46.42347 -23.5...","POLYGON ((357579.81520484 7394795.98601404, 35..."
18361,355030896000246,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 13, 0, 0, 0, 182, ...",8583486.0,96,28.0,LAJEADO,LAJ,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 12, 0, 0, 0, 232, ...",1.000000,18951,"POLYGON ((-46.42247 -23.52123, -46.42214 -23.5...","POLYGON ((357579.81520484 7394795.98601404, 35..."


In [15]:
gdf_setor_distrito.dtypes

cd_original_setor_censitario        object
negative_buffer                     object
cd_identificador_distrito          float64
cd_distrito_municipal               object
cd_identificador_subprefeitura     float64
nm_distrito_municipal               object
sg_distrito_municipal               object
intersection                        object
intersection_pct                   float64
__index_level_0__                    int64
geometry                          geometry
right_geometry                      object
dtype: object

In [16]:
gdf_distrito = load_parquet('distrito_municipal_digested')
gdf_distrito

Gerando query para o arquivo s3://silver-db/dagster/distrito_municipal_digested.parquet...
Query gerada:
SELECT * EXCLUDE (geometry), ST_AsText(ST_GeomFromWKB(geometry)) AS geometry FROM read_parquet("s3://silver-db/dagster/distrito_municipal_digested.parquet")


,cd_identificador_distrito,cd_identificador_subprefeitura,cd_distrito_municipal,nm_distrito_municipal,sg_distrito_municipal,tx_escala,sg_fonte_original,dt_criacao,dt_atualizacao,cd_usuario_atualizacao,cd_tipo_discrepancia,qt_area_quilometro,qt_area_metro,geometry
0,8583394,5,51,MANDAQUI,MAN,1:5000,GEOGSG,2023-11-30Z,2023-11-30T03:00:00Z,NaN,199299399,13.248,1.324786e+07,"POLYGON ((-46.65470 -23.43063, -46.65468 -23.4..."
1,8583395,20,52,MARSILAC,MAR,1:5000,GEOGSG,2023-11-30Z,2023-11-30T03:00:00Z,NaN,199299399,208.196,2.081957e+08,"POLYGON ((-46.60987 -23.90550, -46.60990 -23.9..."
2,8583396,12,32,MOEMA,MOE,1:5000,GEOGSG,2023-11-30Z,2023-11-30T03:00:00Z,NaN,199299399,9.080,9.079516e+06,"POLYGON ((-46.65361 -23.57219, -46.65359 -23.5..."
3,8583445,21,5,ARTUR ALVIM,AAL,1:5000,GEOGSG,2023-11-30Z,2023-11-30T03:00:00Z,NaN,199299399,6.508,6.508361e+06,"POLYGON ((-46.47502 -23.52381, -46.47501 -23.5..."
4,8583472,30,33,IGUATEMI,IGU,1:5000,GEOGSG,2023-11-30Z,2023-11-30T03:00:00Z,NaN,199299399,19.584,1.958356e+07,"POLYGON ((-46.46169 -23.59726, -46.46149 -23.5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,8583448,25,8,BELEM,BEL,1:5000,GEOGSG,2023-11-30Z,2023-11-30T03:00:00Z,NaN,199299399,6.047,6.046777e+06,"POLYGON ((-46.59708 -23.52762, -46.59703 -23.5..."
92,8583480,8,41,JAGUARE,JRE,1:5000,GEOGSG,2023-11-30Z,2023-11-30T03:00:00Z,NaN,199299399,6.580,6.579876e+06,"POLYGON ((-46.73448 -23.54588, -46.73159 -23.5..."
93,8583481,2,42,JARAGUA,JAR,1:5000,GEOGSG,2023-11-30Z,2023-11-30T03:00:00Z,NaN,199299399,28.330,2.833031e+07,"POLYGON ((-46.77026 -23.45337, -46.77025 -23.4..."
94,8583482,23,44,JARDIM HELENA,JDH,1:5000,GEOGSG,2023-11-30Z,2023-11-30T03:00:00Z,NaN,199299399,9.149,9.148559e+06,"POLYGON ((-46.44027 -23.47986, -46.43999 -23.4..."


In [17]:
nm_distrito_selecionado = gdf_distrito.sample(n=1).iloc[0,3]
nm_distrito_selecionado

'PARQUE DO CARMO'

## Centraliza o mapa no distrito

In [18]:
# CartoDB Positron
map = folium.Map(tiles="CartoDB Positron", zoom_start=9)
map

In [19]:
# Para facilitar a edição, defino a tolerância de simplificação das geometrias
simpification_tolerance = 10**-6


Como o folium trabalha apenas na projeção `EPSG4326`, criarei um novo geodataframe convertido para utilizar nos próximos mapas

In [20]:
fol_df = gdf_distrito.copy().to_crs(epsg=4326)
fol_df

,cd_identificador_distrito,cd_identificador_subprefeitura,cd_distrito_municipal,nm_distrito_municipal,sg_distrito_municipal,tx_escala,sg_fonte_original,dt_criacao,dt_atualizacao,cd_usuario_atualizacao,cd_tipo_discrepancia,qt_area_quilometro,qt_area_metro,geometry
0,8583394,5,51,MANDAQUI,MAN,1:5000,GEOGSG,2023-11-30Z,2023-11-30T03:00:00Z,NaN,199299399,13.248,1.324786e+07,"POLYGON ((-46.65470 -23.43063, -46.65468 -23.4..."
1,8583395,20,52,MARSILAC,MAR,1:5000,GEOGSG,2023-11-30Z,2023-11-30T03:00:00Z,NaN,199299399,208.196,2.081957e+08,"POLYGON ((-46.60987 -23.90550, -46.60990 -23.9..."
2,8583396,12,32,MOEMA,MOE,1:5000,GEOGSG,2023-11-30Z,2023-11-30T03:00:00Z,NaN,199299399,9.080,9.079516e+06,"POLYGON ((-46.65361 -23.57219, -46.65359 -23.5..."
3,8583445,21,5,ARTUR ALVIM,AAL,1:5000,GEOGSG,2023-11-30Z,2023-11-30T03:00:00Z,NaN,199299399,6.508,6.508361e+06,"POLYGON ((-46.47502 -23.52381, -46.47501 -23.5..."
4,8583472,30,33,IGUATEMI,IGU,1:5000,GEOGSG,2023-11-30Z,2023-11-30T03:00:00Z,NaN,199299399,19.584,1.958356e+07,"POLYGON ((-46.46169 -23.59726, -46.46149 -23.5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,8583448,25,8,BELEM,BEL,1:5000,GEOGSG,2023-11-30Z,2023-11-30T03:00:00Z,NaN,199299399,6.047,6.046777e+06,"POLYGON ((-46.59708 -23.52762, -46.59703 -23.5..."
92,8583480,8,41,JAGUARE,JRE,1:5000,GEOGSG,2023-11-30Z,2023-11-30T03:00:00Z,NaN,199299399,6.580,6.579876e+06,"POLYGON ((-46.73448 -23.54588, -46.73159 -23.5..."
93,8583481,2,42,JARAGUA,JAR,1:5000,GEOGSG,2023-11-30Z,2023-11-30T03:00:00Z,NaN,199299399,28.330,2.833031e+07,"POLYGON ((-46.77026 -23.45337, -46.77025 -23.4..."
94,8583482,23,44,JARDIM HELENA,JDH,1:5000,GEOGSG,2023-11-30Z,2023-11-30T03:00:00Z,NaN,199299399,9.149,9.148559e+06,"POLYGON ((-46.44027 -23.47986, -46.43999 -23.4..."


Primeiro, adicionarei o contorno geral dos setores censitários para servir de base

In [21]:
fol_df = fol_df.loc[fol_df['nm_distrito_municipal']==nm_distrito_selecionado]
sim_geo = fol_df["geometry"].simplify(tolerance=simpification_tolerance)
geo_j = sim_geo.to_json()
geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {'color': 'navy', 'fillOpacity': 0.2})
geo_j.add_to(map)
map.fit_bounds(map.get_bounds())
map

## Exibir os polígonos dos setores censitários daquele distrito com cores baseadas nas categorias acima

In [22]:
fol_df_2 = gdf_setor.copy().to_crs(epsg=4326)
fol_df_2

,cd_original_setor_censitario,Cod_setor,Domicilio01_SP1_V001,Domicilio01_SP1_V002,Domicilio01_SP1_V012,Domicilio01_SP1_V013,Domicilio01_SP1_V014,Domicilio01_SP1_V017,Domicilio01_SP1_V018,Domicilio01_SP1_V019,Domicilio01_SP1_V020,Domicilio01_SP1_V021,Domicilio01_SP1_V035,Domicilio01_SP1_V039,Domicilio01_SP1_V040,Domicilio01_SP1_V041,missing,supressed,geometry
0,355030801000001,355030801000001,268.0,266.0,266.0,0.0,0.0,266.0,0.0,0.0,0.0,0.0,265.0,0.0,1.0,0.0,False,False,"POLYGON ((-46.56957 -23.56917, -46.57010 -23.5..."
1,355030801000002,355030801000002,306.0,306.0,306.0,0.0,0.0,306.0,0.0,0.0,0.0,0.0,306.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.56876 -23.56854, -46.56883 -23.5..."
2,355030801000003,355030801000003,189.0,189.0,189.0,0.0,0.0,187.0,1.0,1.0,0.0,0.0,189.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.56769 -23.57054, -46.56780 -23.5..."
3,355030801000004,355030801000004,181.0,181.0,181.0,0.0,0.0,181.0,0.0,0.0,0.0,0.0,181.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.56747 -23.57330, -46.56821 -23.5..."
4,355030801000005,355030801000005,240.0,240.0,240.0,0.0,0.0,240.0,0.0,0.0,0.0,0.0,240.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.57120 -23.57313, -46.57127 -23.5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18948,355030896000243,355030896000243,96.0,96.0,96.0,0.0,0.0,96.0,0.0,0.0,0.0,0.0,96.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.38858 -23.54386, -46.38858 -23.5..."
18949,355030896000244,355030896000244,54.0,54.0,54.0,0.0,0.0,52.0,0.0,0.0,0.0,2.0,54.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.38815 -23.54508, -46.38816 -23.5..."
18950,355030896000245,355030896000245,87.0,87.0,87.0,0.0,0.0,15.0,0.0,0.0,0.0,72.0,87.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.42368 -23.52080, -46.42347 -23.5..."
18951,355030896000246,355030896000246,38.0,38.0,38.0,0.0,0.0,18.0,0.0,0.0,0.0,20.0,38.0,0.0,0.0,0.0,False,False,"POLYGON ((-46.42247 -23.52123, -46.42214 -23.5..."


In [23]:
gdf_setor_distrito_filtrado = gdf_setor_distrito[gdf_setor_distrito['nm_distrito_municipal']==nm_distrito_selecionado]
filter_setor = fol_df_2['cd_original_setor_censitario'].isin(gdf_setor_distrito_filtrado['cd_original_setor_censitario'])

fol_df_2 = fol_df_2.loc[filter_setor]
sim_geo = fol_df_2["geometry"].simplify(tolerance=simpification_tolerance)
geo_j = sim_geo.to_json()
geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {'color': 'orange', 'fillOpacity': 0.1})
geo_j.add_to(map)
map.fit_bounds(map.get_bounds())
map

### Segundo modo (.explore)

In [24]:
base_map = fol_df.explore(color='navy', tiles="CartoDB Positron")
m = fol_df_2.explore(color='orange', m=base_map)
m

## Exibir informações adicionais sobre o setor quando seu polígono for clicado no mapa

## Permitir a  a inclusão ou remoção de outras camadas relacionadas a um determinado setor, com base nos assets `intersection_setor_*`